## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_2 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o2/o2_bincore8.csv", index_col=0)
print(bin8_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_2.reset_index(inplace=True, drop=True)

print('shape -', bin8_2.shape)
print('reset_index 완료')
print('input data shape')
bin8_2.head()

(28744700, 2)
shape - (28744700, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_2['label'].value_counts())

256
0    28706035
1       38665
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_2[bin8_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_2', len(ls_idx_bin))

# loc 로 수정필요
bin8_2_Ngram = bin8_2.loc[ls_idx_bin,:].copy()

618640
나머지 0
최종 길이 618640
bin8_2 618640


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

61202.0
0 16865252
1000 14634078
2000 1006465
3000 24740403
4000 23499391
5000 12708635
6000 15791542
7000 4126182
8000 1159586
9000 1229493
10000 6220577
11000 25936301
12000 23484260
13000 10125522
14000 27602874
15000 5938225
16000 11661521
17000 26125788
18000 23113329
19000 25555242
20000 13180624
21000 17617143
22000 16330579
23000 1409415
24000 16039669
25000 9897156
26000 21808436
27000 17181023
28000 15851753
29000 23149876
30000 1590054
31000 3905898
32000 25393602
33000 26700890
34000 26347897
35000 15162697
36000 1460018
37000 3721894
38000 15595297
39000 5234517
40000 6130607
41000 13215244
42000 2791882
43000 3928689
44000 12624766
45000 19939218
46000 26713682
47000 23128245
48000 26837562
49000 2519368
50000 13506565
51000 17028086
52000 17319918
53000 26894612
54000 6521539
55000 10571784
56000 14837922
57000 17547632
58000 12951286
59000 17765565
60000 22563033
61000 19692180
완료
61202


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_2['label'].value_counts()[1])

61202
38665


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_2_Ngram])
final.shape

(1958464, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_2_onehot_Ngram = pd.concat([final['label'], bc8_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_2_onehot_Ngram.shape)

원핫인코딩완료
(1958464, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_2 = bc8_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_2 = bc8_2_onehot_Ngram['label'].to_numpy()
print(x_bc8_2.shape, x_bc8_2.shape)

x_bc8_2 = x_bc8_2.reshape(-1, right_idx, x_bc8_2.shape[1])
y_bc8_2 = y_bc8_2.reshape(-1, right_idx, 1)

print(x_bc8_2.shape, y_bc8_2.shape)

(1958464, 256) (1958464, 256)
(122404, 16, 256) (122404, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_2.shape[0])

x_bc8_2 = x_bc8_2[p]
y_bc8_2 = y_bc8_2[p]

print(x_bc8_2.shape, y_bc8_2.shape)

(122404, 16, 256) (122404, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_2, y_bc8_2):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_2[train],
               y_bc8_2[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_2[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_2[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 60s 545us/sample - loss: 0.0242 - accuracy: 0.9931
Epoch 2/10
110163/110163 [==============================] - 49s 446us/sample - loss: 0.0091 - accuracy: 0.9970
Epoch 3/10
110163/110163 [==============================] - 60s 545us/sample - loss: 0.0075 - accuracy: 0.9976
Epo

accuracy_score 0.9986673882852708
recall_score 0.9724770642201835
precision_score 0.9647904191616766
f1_score 0.9686184922447998
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 95s 858us/sample - loss: 0.0037 - accuracy: 0.9988
Epoch 2/10
110163/110163 [==============================] - 73s 660us/sample - loss: 0.0035 - 

accuracy_score 0.9989226778857937
recall_score 0.9715167339188227
precision_score 0.9782504780114722
f1_score 0.9748719780874121
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 96s 868us/sample - loss: 0.0026 - accuracy: 0.9992
Epoch 2/10
110163/110163 [==============================] - 65s 589us/sample - loss: 0.0025 - 

accuracy_score 0.9991218037742015
recall_score 0.9787183165949307
precision_score 0.9801245210727969
f1_score 0.979420914094281
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 93s 847us/sample - loss: 0.0020 - accuracy: 0.9994
Epoch 2/10
110163/110163 [==============================] - 83s 757us/sample - loss: 0.0019 - a

accuracy_score 0.9991115921901805
recall_score 0.9805548968460991
precision_score 0.9782351549562337
f1_score 0.9793936522974894
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 90s 815us/sample - loss: 0.0016 - accuracy: 0.9995
Epoch 2/10
110164/110164 [==============================] - 72s 652us/sample - loss: 0.0015 - 

...

accuracy_score 0.9992800245098039
recall_score 0.9834928229665072
precision_score 0.9827874731054267
f1_score 0.9831400215233768
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 78s 704us/sample - loss: 0.0012 - accuracy: 0.9996
Epoch 2/10
110164/110164 [==============================] - 63s 575us/sample - loss: 0.0011 - 

accuracy_score 0.9992544934640523
recall_score 0.9779498006098991
precision_score 0.9876806443970623
f1_score 0.9827911362564827
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 69s 624us/sample - loss: 0.0011 - accuracy: 0.9997
Epoch 2/10
110164/110164 [==============================] - 65s 587us/sample - loss: 0.0010 - 

accuracy_score 0.9993259803921568
recall_score 0.9822686422152053
precision_score 0.9856202778454789
f1_score 0.983941605839416
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 84s 763us/sample - loss: 9.6199e-04 - accuracy: 0.9997
Epoch 2/10
110164/110164 [==============================] - 65s 586us/sample - loss: 8.4614

accuracy_score 0.9993923611111111
recall_score 0.9836218998596162
precision_score 0.9884787209028921
f1_score 0.9860443297760056
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 82s 747us/sample - loss: 9.3271e-04 - accuracy: 0.9998
Epoch 2/10
110164/110164 [==============================] - 69s 628us/sample - loss: 8.119

accuracy_score 0.9995302287581699
recall_score 0.988909863142992
precision_score 0.9893767705382436
f1_score 0.9891432617417983

K-fold cross validation Accuracy: [0.9986673882852708, 0.9989226778857937, 0.9991218037742015, 0.9991115921901805, 0.9990808823529411, 0.9992800245098039, 0.9992544934640523, 0.9993259803921568, 0.9993923611111111, 0.9995302287581699]

K-fold cross validation Recall: [0.9724770642201835, 0.9715167339188227, 0.9787183165949307, 0.9805548968460991, 0.9787532103665655, 0.9834928229665072, 0.9779498006098991, 0.9822686422152053, 0.9836218998596162, 0.988909863142992]

K-fold cross validation Precision: [0.9647904191616766, 0.9782504780114722, 0.9801245210727969, 0.9782351549562337, 0.9792104648446625, 0.9827874731054267, 0.9876806443970623, 0.9856202778454789, 0.9884787209028921, 0.9893767705382436]

K-fold cross validation F1-Score: [0.9686184922447998, 0.9748719780874121, 0.979420914094281, 0.9793936522974894, 0.9789817842129845, 0.9831400215233768, 0.982791136

## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9991687432723682
10-Fold Cross_validation. Recall : 0.9798263250740821
10-Fold Cross_validation. Precision : 0.9814554924835945
10-Fold Cross_validation. F1-Score : 0.9806347176074046


In [13]:
model1.save('gcc8_bin_core_s16_h24_o2.h5')
print('save 완료')

save 완료
